In [1]:
from tensorflow.keras import layers
from tensorflow.keras import losses

In [2]:
#Import libraries
import pandas as pd
from sklearn.model_selection import train_test_split

from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.layers import Input, Dropout, Dense

In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [4]:
!pip install --upgrade simple_elmo

Requirement already up-to-date: simple_elmo in /usr/local/lib/python3.7/dist-packages (0.8.0)


In [5]:
!wget https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x1024_128_2048cnn_1xhighway/elmo_2x1024_128_2048cnn_1xhighway_options.json
!wget https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x1024_128_2048cnn_1xhighway/elmo_2x1024_128_2048cnn_1xhighway_weights.hdf5

--2021-06-03 14:57:43--  https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x1024_128_2048cnn_1xhighway/elmo_2x1024_128_2048cnn_1xhighway_options.json
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.237.56
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.237.56|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 336 [application/json]
Saving to: ‘elmo_2x1024_128_2048cnn_1xhighway_options.json’

elmo_2x1024_128_204 100%[===================>]     336  --.-KB/s    in 0s      

2021-06-03 14:57:43 (20.1 MB/s) - ‘elmo_2x1024_128_2048cnn_1xhighway_options.json’ saved [336/336]

--2021-06-03 14:57:43--  https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x1024_128_2048cnn_1xhighway/elmo_2x1024_128_2048cnn_1xhighway_weights.hdf5
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.237.56
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.237.56|:443... conne

In [6]:
!mv elmo_2x1024_128_2048cnn_1xhighway_options.json options.json

In [7]:
from simple_elmo import ElmoModel
model = ElmoModel()


In [8]:
#model.load('/content',max_batch_size=32)
model.load('/content',max_batch_size=16)

2021-06-03 14:57:46,228 : INFO : Loading model from /content...
2021-06-03 14:57:46,230 : INFO : No vocabulary file found in the model.
2021-06-03 14:57:46,232 : INFO : No model.hdf5 file found. Using /content/elmo_2x1024_128_2048cnn_1xhighway_weights.hdf5 as a model file.
2021-06-03 14:57:46,233 : INFO : No vocabulary file provided; using special tokens only.
2021-06-03 14:57:46,235 : INFO : We will cache the vocabulary of 3 tokens.
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:909: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please u

'The model is now loaded.'

In [9]:
df = pd.read_csv('/content/drive/MyDrive/ADV RE/app_reviews_all_annotated2.csv')
df = df[['review', 'argument_cat', 'decision_cat']]

# Remove missing rows
df = df.dropna()

df = df.groupby('argument_cat').filter(lambda x : len(x) > 1)
df = df.groupby('decision_cat').filter(lambda x : len(x) > 1)

# Convert to numeric for bert
df['Argument'] = pd.Categorical(df['argument_cat'])
df['Decision'] = pd.Categorical(df['decision_cat'])
df['argument_cat'] = df['Argument'].cat.codes
df['decision_cat'] = df['Decision'].cat.codes

2021-06-03 14:57:47,166 : INFO : NumExpr defaulting to 2 threads.


In [10]:
df2 = df.drop(columns=['Argument','Decision'])
df2

,review,argument_cat,decision_cat
5,Everyone should use it Full Review,2,0
37,Improve face detection please !!! Full Rev...,2,4
192,I think we just need more updates Full Rev...,2,4
201,if you're looking for an alternative for Pho...,1,3
221,it's the best photo editor app i've ever see...,1,4
...,...,...,...
45986,OMG IT'S SO AWESOME YOU SHOULD TRY IT Full...,1,3
46051,I litterly love this game i use to play this...,1,0
46076,"Please make more levels. Other wise, it's go...",1,4
46077,awesome game worth the money! Full Review,1,1


In [11]:
train_df, test_df = train_test_split(df2, test_size = 0.1, stratify=df2[['argument_cat']])

In [12]:
x_train = train_df['review']
y_train = train_df['argument_cat']
x_test = test_df['review']
y_test = test_df['argument_cat']

In [13]:
import pickle

PATH = '/content/drive/MyDrive/ADV RE/'

try:
    x_train_embeddings = pickle.load(open(PATH + "x_train_embeddings.pickle", "rb" ))
except (OSError, IOError) as e:
    x_train_embeddings = model.get_elmo_vector_average(x_train.to_list())
    pickle.dump(x_train_embeddings, open(PATH + "x_train_embeddings.pickle",'wb'))


2021-06-03 14:57:48,178 : INFO : Warming up ELMo on 16 sentences...
2021-06-03 14:57:52,222 : INFO : Warming up finished.
2021-06-03 14:57:52,243 : INFO : Texts in the current batch: 16
2021-06-03 14:57:53,522 : INFO : Texts in the current batch: 16
2021-06-03 14:57:54,449 : INFO : Texts in the current batch: 16
2021-06-03 14:57:55,484 : INFO : Texts in the current batch: 16
2021-06-03 14:57:57,167 : INFO : Texts in the current batch: 16
2021-06-03 14:57:58,855 : INFO : Texts in the current batch: 16
2021-06-03 14:58:00,461 : INFO : Texts in the current batch: 16
2021-06-03 14:58:01,748 : INFO : Texts in the current batch: 16
2021-06-03 14:58:02,647 : INFO : Texts in the current batch: 16
2021-06-03 14:58:04,208 : INFO : Texts in the current batch: 16
2021-06-03 14:58:05,715 : INFO : Texts in the current batch: 16
2021-06-03 14:58:06,745 : INFO : Texts in the current batch: 16
2021-06-03 14:58:08,196 : INFO : Texts in the current batch: 16
2021-06-03 14:58:09,284 : INFO : Texts in the 

In [14]:
#vectors = model.get_elmo_vector_average(x_train.to_list()[0:128])
#x_train_embeddings = model.get_elmo_vector_average(x_train.to_list())

In [15]:
x_train_embeddings.shape

(2488, 256)

In [16]:
try:
    x_test_embeddings = pickle.load(open(PATH + "x_test_embeddings.pickle", "rb" ))
except (OSError, IOError) as e:
    x_test_embeddings = model.get_elmo_vector_average(x_test.to_list())
    pickle.dump(x_test_embeddings, open(PATH + "x_test_embeddings.pickle",'wb'))

2021-06-03 15:01:13,187 : INFO : Warming up ELMo on 16 sentences...
2021-06-03 15:01:14,635 : INFO : Warming up finished.
2021-06-03 15:01:14,648 : INFO : Texts in the current batch: 16
2021-06-03 15:01:15,523 : INFO : Texts in the current batch: 16
2021-06-03 15:01:16,341 : INFO : Texts in the current batch: 16
2021-06-03 15:01:17,628 : INFO : Texts in the current batch: 16
2021-06-03 15:01:18,578 : INFO : Texts in the current batch: 16
2021-06-03 15:01:21,838 : INFO : Texts in the current batch: 16
2021-06-03 15:01:23,024 : INFO : Texts in the current batch: 16
2021-06-03 15:01:24,263 : INFO : Texts in the current batch: 16
2021-06-03 15:01:25,412 : INFO : Texts in the current batch: 16
2021-06-03 15:01:26,329 : INFO : Texts in the current batch: 16
2021-06-03 15:01:27,899 : INFO : Texts in the current batch: 16
2021-06-03 15:01:28,991 : INFO : Texts in the current batch: 16
2021-06-03 15:01:30,436 : INFO : Texts in the current batch: 16
2021-06-03 15:01:31,358 : INFO : Texts in the 

In [17]:
#test_vectors = model.get_elmo_vector_average(x_test.to_list()[0:128])
#x_test_embeddings = model.get_elmo_vector_average(x_test.to_list())

In [18]:
inputs = tf.keras.Input(shape=256)
x = tf.keras.layers.Dense(32, activation=tf.nn.relu)(inputs)

arg = tf.keras.layers.Dense(4, activation=tf.nn.softmax, name='argument')(x)
dec = tf.keras.layers.Dense(5, activation=tf.nn.softmax, name='decision')(x)
outputs = {'argument': arg, 'decision': dec}

my_model = tf.keras.Model(inputs=inputs, outputs=outputs, name='ELMo_For_App_Review_Classification')


#arg = Dense(units=len(df.argument_cat.value_counts()), kernel_initializer=TruncatedNormal(stddev=conf.initializer_range), name='argument')(pooledOutput)
#dec = Dense(units=len(df.decision_cat.value_counts()), kernel_initializer=TruncatedNormal(stddev=conf.initializer_range), name='decision')(pooledOutput)
#outputs = {'argument': arg, 'decision': dec}

my_model.summary()

Model: "ELMo_For_App_Review_Classification"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 32)           8224        input_1[0][0]                    
__________________________________________________________________________________________________
argument (Dense)                (None, 4)            132         dense[0][0]                      
__________________________________________________________________________________________________
decision (Dense)                (None, 5)            165         dense[0][0]                      
Total params: 8,521
Trainable params: 8,521
Non-trainable params:

In [19]:
optimizer = Adam(learning_rate=5e-05, epsilon=1e-08, decay=0.01, clipnorm=1.0)
loss = {'argument': CategoricalCrossentropy(from_logits = True), 'decision': CategoricalCrossentropy(from_logits = True)}
metric = {'argument': CategoricalAccuracy('accuracy'), 'decision': CategoricalAccuracy('accuracy')}
my_model.compile(optimizer = optimizer, loss = loss, metrics = metric)

In [20]:
history = my_model.fit(x=x_train_embeddings, y={'argument': to_categorical(train_df.argument_cat), 'decision': to_categorical(train_df.decision_cat)},
                    validation_split=0.1, batch_size=64, epochs=15)

Train on 2239 samples, validate on 249 samples
Epoch 1/15
2239/2239 [==============================] - 0s 146us/sample - loss: 2.9806 - argument_loss: 1.3714 - decision_loss: 1.6092 - argument_accuracy: 0.3042 - decision_accuracy: 0.0451 - val_loss: 2.9739 - val_argument_loss: 1.3654 - val_decision_loss: 1.6086 - val_argument_accuracy: 0.6145 - val_decision_accuracy: 0.0321
Epoch 2/15
1152/2239 [==============>...............] - ETA: 0s - loss: 2.9724 - argument_loss: 1.3639 - decision_loss: 1.6085 - argument_accuracy: 0.6606 - decision_accuracy: 0.0538

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


2239/2239 [==============================] - 0s 56us/sample - loss: 2.9699 - argument_loss: 1.3618 - decision_loss: 1.6081 - argument_accuracy: 0.7075 - decision_accuracy: 0.0594 - val_loss: 2.9644 - val_argument_loss: 1.3568 - val_decision_loss: 1.6077 - val_argument_accuracy: 0.8193 - val_decision_accuracy: 0.0723
Epoch 3/15
2239/2239 [==============================] - 0s 51us/sample - loss: 2.9609 - argument_loss: 1.3537 - decision_loss: 1.6073 - argument_accuracy: 0.8392 - decision_accuracy: 0.1000 - val_loss: 2.9566 - val_argument_loss: 1.3496 - val_decision_loss: 1.6070 - val_argument_accuracy: 0.8554 - val_decision_accuracy: 0.1124
Epoch 4/15
2239/2239 [==============================] - 0s 63us/sample - loss: 2.9536 - argument_loss: 1.3468 - decision_loss: 1.6067 - argument_accuracy: 0.8665 - decision_accuracy: 0.1335 - val_loss: 2.9502 - val_argument_loss: 1.3435 - val_decision_loss: 1.6067 - val_argument_accuracy: 0.8434 - val_decision_accuracy: 0.1486
Epoch 5/15
2239/2239 [==

In [21]:
#my_model.compile(optimizer='sgd',
#              loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

In [22]:
#epochs = 10
##history = my_model.fit(
##    vectors,
##    epochs=epochs,y=y_train[0:128])
#
#history = my_model.fit(
#    vectors,
#    epochs=epochs,y=y_train)


In [24]:
testArg = to_categorical(test_df['argument_cat'], 4)
testDec = to_categorical(test_df['decision_cat'])

modelEval = my_model.evaluate(x=x_test_embeddings,y={'argument': testArg, 'decision': testDec})

In [30]:
for x in zip(my_model.metrics_names,modelEval):
  print(x)

('loss', 2.9115040775670904)
('argument_loss', 1.3071111)
('decision_loss', 1.6044036)
('argument_accuracy', 0.84476537)
('decision_accuracy', 0.41155234)


In [ ]:
#loss, accuracy = my_model.evaluate(x=test_vectors,y=y_test[0:128])
#loss, accuracy = my_model.evaluate(x=x_test_embeddings,y=y_test)

In [ ]:
#print("Test Loss: " + str(loss))
#print("Test Accuracy: " + str(accuracy))